In [1]:
!pip install dash plotly pandas

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import dash
from dash import html, dcc, Input, Output
import plotly.graph_objects as go
import pandas as pd

try:
    df = pd.read_parquet('master_stock_data.parquet')
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: 'master_stock_data.parquet' not found.")
    try:
        df = pd.read_csv('master_stock_data.csv')
        df['Date'] = pd.to_datetime(df['Date']) 
        print("CSV Data loaded successfully!")
    except:
        print("Critical Error: No data file found. Run Step 1 again.")


app = dash.Dash(__name__)
all_tickers = sorted(df['Ticker'].unique())


app.layout = html.Div([
    html.H1("Trading Strategy Performance", style={'textAlign': 'center', 'fontFamily': 'Arial'}),
    
    html.Div([
        html.Label("Select Asset:", style={'fontWeight': 'bold'}),
        dcc.Dropdown(
            id='ticker-dropdown',
            options=all_tickers,
            value=all_tickers[0],
            clearable=False
        )
    ], style={'width': '50%', 'margin': 'auto', 'padding': '20px'}),
    
    dcc.Graph(id='performance-graph')
])

# Logic
@app.callback(
    Output('performance-graph', 'figure'),
    Input('ticker-dropdown', 'value')
)
def update_graph(selected_ticker):
    # 1. Filter data
    dff = df[df['Ticker'] == selected_ticker].copy()
    dff = dff.sort_values('Date') 
    
# Calculate Growth of $10,000 (Buy & Hold Benchmark)
    initial_investment = 10000
    start_price = dff['Close'].iloc[0]
    
# Formula: (Current Price / Start Price) * $10,000
    dff['Equity'] = (dff['Close'] / start_price) * initial_investment

    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=dff['Date'], 
        y=dff['Equity'],
        mode='lines',
        name='Buy & Hold Strategy',
        line=dict(color='#00CC96', width=2) 
    ))

    fig.add_hline(y=10000, line_dash="dash", line_color="gray", annotation_text="Start ($10k)")

    fig.update_layout(
        title=f"Growth of $10,000 Invested in {selected_ticker}",
        xaxis_title="Date",
        yaxis_title="Account Value ($)",
        template="plotly_white",
        hovermode="x unified"
    )
    return fig

if __name__ == '__main__':
    print("Starting Dashboard...")
    app.run(debug=True)

Data loaded successfully!
Starting Dashboard...
